# Predicted Component Scores from Composite

We're only given composite scores along with component ranks of each school, let's try to approximate the component scores given the composite formula:
```
composite = (0.5 * equity) + (0.25 * excellence) + (0.25 * efficiency)
```

## Linear Model
If we assume a linear relationship between ranks (0 is the worst) and component scores we have the additional relationships:
```
rank_normalized =  rank / total_schools

equity = equity_scaling_factor * equity_rank_normalized
excellence = excellence_scaling_factor * excellence_rank_normalized
efficiency = efficiency_scaling_factor * efficiency_rank_normalized
```

Combinging these we arrive at the equation we're trying to solve for:
```
composite = (0.5 * equity_scaling_factor * equity_rank_normalized) 
    + (0.25 * excellence_scaling_factor * excellence_rank_normalized) 
    + (0.25 * efficiency_scaling_factor * efficiency_rank_normalized)

composite = beta_0 
    + (beta_1 * equity_rank_normalized) 
    + (beta_2 * excellence_rank_normalized) 
    + (beta_3 * efficiency_rank_normalized)
```

Solving for the `betas` allows us to then approcimate the individual component scores.

## Power Model
Although we don't know the shape of the distribution of each component score, we do we see that the composite score distribution is only mostly linear. The head and tail have increased slopes which is better fitted with a power model, if we assume the composite score distrubution is also representative of the component score distributions we can instead fit a power model solving for `p`s:
```
composite = (0.5 * equity_rank_normalized ** p_equity) 
    + (0.25 * excellence_rank_normalized ** p_excellence) 
    + (0.25 * efficiency_rank_normalized ** p_efficiency)
```

## Skewed Normal Model
Looking at the results, the head and tail of each ranks have much higher errors (even with a power model). This is likely due to the fact that the rank is linear, but the actual is likely a skewed normal distribution.

Consider "excellence" which is a measure of academics, most schools likely perform similarly clustered in the "middle" with less less lower and higher performing schools forming a bell curve or skewed bell curve. This is likely for all component scores so lets use this assumption to fit a model as well.

Like the rest, lets use a regression to fit a normal distribution for each component score.

## Results
Skewed normal does slightly better especially for MSE.

Linear Model Errors:
- MAE: 5.1746
- MSE: 54.1848
- RMSE: 7.3610

Power Model Errors
- MAE: 4.7108
- MSE: 45.3876
- RMSE: 6.7370

Skewed Normal Model Errors
- MAE: 4.1992
- MSE: 37.5471
- RMSE: 6.1276


## Prep the Data

In [89]:
%run notebooks/Setup.ipynb

import polars
import numpy
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.base import BaseEstimator, RegressorMixin
from scipy.optimize import minimize
from scipy.stats import skewnorm

In [90]:
# source data we're going to be working with
composite_df = polars.read_csv(workspace_path.joinpath('data/processed/composite_scores_raw.csv'))
composite_df

# normalize the ranks, the max is the same across components
max_rank = composite_df['equity_rank'].max() + 2

normalized_df = composite_df\
    .select(['school_name', 'composite_score', 'equity_rank', 'excellence_rank', 'efficiency_rank'])\
    .with_columns([
        ((composite_df['equity_rank'] + 1) / max_rank).alias('equity_rank_normalized'),
        ((composite_df['excellence_rank'] + 1) / max_rank).alias('excellence_rank_normalized'),
        ((composite_df['efficiency_rank'] + 1) / max_rank).alias('efficiency_rank_normalized'),
        (composite_df['composite_score'] / 100).alias('composite_normalized'),
    ])

normalized_df

school_name,composite_score,equity_rank,excellence_rank,efficiency_rank,equity_rank_normalized,excellence_rank_normalized,efficiency_rank_normalized,composite_normalized
str,f64,i64,i64,i64,f64,f64,f64,f64
"""Washington (George) High""",72.91,81,52,98,0.803922,0.519608,0.970588,0.7291
"""Presidio Middle""",51.16,52,44,94,0.519608,0.441176,0.931373,0.5116
"""Lafayette Elementary""",23.61,7,96,48,0.078431,0.95098,0.480392,0.2361
"""Alamo Elementary""",14.13,6,74,35,0.068627,0.735294,0.352941,0.1413
"""Argonne Elementary""",11.46,4,75,40,0.04902,0.745098,0.401961,0.1146
…,…,…,…,…,…,…,…,…
"""Milk (Harvey) Civil Rights Ele…",21.5,9,73,71,0.098039,0.72549,0.705882,0.215
"""Everett Middle""",15.2,79,1,4,0.784314,0.019608,0.04902,0.152
"""Lilienthal (Claire) Elementary""",40.88,31,87,61,0.313725,0.862745,0.607843,0.4088


## Model Eval

In [91]:
# retrun the errors of the predictions to evaluate the model
def errors(predicted_df):
    mae = mean_absolute_error(predicted_df['composite_score'], predicted_df['composite_score_predicted'])
    mse = mean_squared_error(predicted_df['composite_score'], predicted_df['composite_score_predicted'])
    
    print(f'MAE: {mae:.4f}')
    print(f'MSE: {mse:.4f}')
    print(f'RMSE: {numpy.sqrt(mse):.4f}')


## Linear Model

In [92]:
# setup the regressions
x = normalized_df.select(['equity_rank_normalized', 'excellence_rank_normalized', 'efficiency_rank_normalized'])\
    .to_numpy()
y = normalized_df['composite_normalized'].to_numpy()

# fit
model = LinearRegression(fit_intercept=False)
model.fit(x, y)
beta_0 = model.intercept_
beta_1, beta_2, beta_3 = model.coef_

beta_0, beta_1, beta_2, beta_3

(0.0,
 np.float64(0.3976023674406394),
 np.float64(0.15368622846912555),
 np.float64(0.21555430851378687))

In [93]:
# predict and eval
equity_scaling_factor = beta_1 / 0.5
excellence_scaling_factor = beta_2 / 0.25
efficiency_scaling_factor = beta_3 / 0.25

linear_predictions = normalized_df.with_columns([
    (equity_scaling_factor * normalized_df['equity_rank_normalized'] * 100).alias('equity_score_predicted'),
    (excellence_scaling_factor * normalized_df['excellence_rank_normalized'] * 100).alias('excellence_score_predicted'),
    (efficiency_scaling_factor * normalized_df['efficiency_rank_normalized'] * 100).alias('efficiency_score_predicted'),
])

linear_predictions = linear_predictions.with_columns(
    (
        0.5 * linear_predictions['equity_score_predicted'] +
        0.25 * linear_predictions['excellence_score_predicted'] +
        0.25 * linear_predictions['efficiency_score_predicted'] -
        beta_0
    ).alias('composite_score_predicted')
)

errors(linear_predictions)

MAE: 5.3005
MSE: 56.2090
RMSE: 7.4973


## Power Model

In [176]:
def fit(params, data):
    b_equity, b_excellence, b_efficiency, p_equity, p_excellence, p_efficiency = params
    
    # estimate the parts and composite
    data['s_equity'] = b_equity * data['equity_rank_normalized'] ** p_equity
    data['s_excellence'] = b_excellence * data['excellence_rank_normalized'] ** p_excellence
    data['s_efficiency'] = b_efficiency * data['efficiency_rank_normalized'] ** p_efficiency
    
    data['composite_estimated'] = (
        0.5 * data['s_equity'] +
        0.25 * data['s_excellence'] +
        0.25 * data['s_efficiency']
    )
    
    # return sum squared of errors to minimize
    sse = numpy.sum((data['composite_normalized'] - data['composite_estimated']) ** 2)
    
    return sse

In [177]:
initial_params = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
bounds = [(-100, 100), (-100, 100), (-100, 100), (0.01, 20), (00.01, 20), (00.01, 20)]

result = minimize(
    fit,
    initial_params,
    args=(normalized_df.to_pandas(),),
    bounds=bounds,
    method='L-BFGS-B'
)

# extract the powers
b_equity, b_excellence, b_efficiency, p_equity, p_excellence, p_efficiency = result.x
result

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.4231055467648279
        x: [ 8.993e-01  9.060e-01  1.064e+00  1.141e+00  1.240e+00
             2.956e+00]
      nit: 23
      jac: [ 3.686e-06  2.048e-06 -3.830e-07 -9.270e-07 -3.442e-07
            -4.996e-08]
     nfev: 175
     njev: 25
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>

In [178]:
# predict and eval
power_predictions = normalized_df.with_columns([
    (b_equity * normalized_df['equity_rank_normalized'] ** p_equity * 100).alias('equity_score_predicted'),
    (b_excellence * normalized_df['excellence_rank_normalized'] ** p_excellence * 100).alias('excellence_score_predicted'),
    (b_efficiency * normalized_df['efficiency_rank_normalized'] ** p_efficiency * 100).alias('efficiency_score_predicted'),
])

power_predictions = power_predictions.with_columns(
    (
        0.5 * power_predictions['equity_score_predicted'] +
        0.25 * power_predictions['excellence_score_predicted'] +
        0.25 * power_predictions['efficiency_score_predicted']
    ).alias('composite_score_predicted')
)

errors(power_predictions)

MAE: 4.4308
MSE: 41.8916
RMSE: 6.4724


## Skewed Normal Distribution Model

In [267]:
def fit(params, data):
    a_equity, a_excellence, a_efficiency, l_equity, l_excellence, l_efficiency, s_equity, s_excellence, s_efficiency = params
    
    # estimate the parts and composite
    data['s_equity'] = skewnorm.ppf(data['equity_rank_normalized'], a_equity, loc=l_equity, scale=s_equity)
    data['s_excellence'] = skewnorm.ppf(data['excellence_rank_normalized'], a_excellence, loc=l_excellence, scale=s_excellence)
    data['s_efficiency'] = skewnorm.ppf(data['efficiency_rank_normalized'], a_efficiency, loc=l_efficiency, scale=s_efficiency)

    data['composite_estimated'] = (
        0.5 * data['s_equity'] +
        0.25 * data['s_excellence'] +
        0.25 * data['s_efficiency']
    )
    
    # return sum squared of errors to minimize
    sse = numpy.sum((data['composite_score'] - data['composite_estimated']) ** 2)
    
    return sse

In [268]:
initial_params = [0.0, 0.0, 0.0, 50.0, 50.0, 50.0, 10.0, 10.0, 10.0]
bounds = [
    (-20, 20), 
    (-20, 20),
    (-20, 20),
    (30, 70),
    (30, 70),
    (30, 70),
    (1, 30),
    (1, 30),
    (1, 30)
]

result = minimize(
    fit,
    initial_params,
    args=(normalized_df.to_pandas(),),
    bounds=bounds,
    method='L-BFGS-B'
)

a_equity, a_excellence, a_efficiency, l_equity, l_excellence, l_efficiency, s_equity, s_excellence, s_efficiency = result.x
result

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 1835.7595418624173
        x: [-3.742e-07 -6.395e-07 -6.441e-07  3.762e+01  3.638e+01
             3.638e+01  3.000e+01  3.000e+01  3.000e+01]
      nit: 45
      jac: [ 1.683e-03  9.095e-04  6.821e-04  1.819e-04  9.095e-05
             4.547e-05 -1.190e+02 -3.644e+01 -6.072e+01]
     nfev: 670
     njev: 67
 hess_inv: <9x9 LbfgsInvHessProduct with dtype=float64>

In [271]:
# predict and eval
normal_predictions = normalized_df.with_columns(
    # need to clip between possible bounds
    equity_score_predicted = numpy.clip(skewnorm.ppf(normalized_df['equity_rank_normalized'], a_equity, loc=l_equity, scale=s_equity), 0, 100),
    excellence_score_predicted = numpy.clip(skewnorm.ppf(normalized_df['excellence_rank_normalized'], a_excellence, loc=l_excellence, scale=s_excellence), 0, 100),
    efficiency_score_predicted = numpy.clip(skewnorm.ppf(normalized_df['efficiency_rank_normalized'], a_efficiency, loc=l_efficiency, scale=s_efficiency), 0, 100)
)

normal_predictions = normal_predictions.with_columns(
    composite_score_predicted = (
        0.5 * normal_predictions['equity_score_predicted'] +
        0.25 * normal_predictions['excellence_score_predicted'] +
        0.25 * normal_predictions['efficiency_score_predicted']
    )
)

errors(normal_predictions)

MAE: 4.1992
MSE: 37.5471
RMSE: 6.1276


## Save

Combine with original file and save.

In [272]:
enrollment = polars.read_csv(workspace_path.joinpath('data/raw/rai/composite_scores_doc.csv'))\
    .drop(['Composite Score'])\
    .rename({'School Name': 'school_name'})

normal_predictions = normal_predictions\
    .select(['school_name', 'equity_score_predicted', 'excellence_score_predicted', 'efficiency_score_predicted', 'composite_score_predicted'])\
    .rename({
        'equity_score_predicted': 'equity_score_p',
        'excellence_score_predicted': 'excellence_score_p',
        'efficiency_score_predicted': 'efficiency_score_p',
        'composite_score_predicted': 'composite_score_p'
    })

# merge with original
combined_predictions = composite_df\
    .select(['school_name', 'composite_score', 'equity_rank', 'excellence_rank', 'efficiency_rank'])\
    .join(
        enrollment,
        on='school_name', 
        how='left'
    )\
    .join(
        normal_predictions, 
        on='school_name', 
        how='left'
    )

combined_predictions.write_csv(workspace_path.joinpath('data/processed/component_scores.csv'))
combined_predictions


school_name,composite_score,equity_rank,excellence_rank,efficiency_rank,Enrollment,equity_score_p,excellence_score_p,efficiency_score_p,composite_score_p
str,f64,i64,i64,i64,i64,f64,f64,f64,f64
"""Washington (George) High""",72.91,81,52,98,2091,63.294783,37.851138,93.061238,64.375485
"""Presidio Middle""",51.16,52,44,94,996,39.098491,31.936443,80.958686,47.773028
"""Lafayette Elementary""",23.61,7,96,48,468,0.0,86.009098,34.900858,30.227489
"""Alamo Elementary""",14.13,6,74,35,390,0.0,55.243183,25.054181,20.074341
"""Argonne Elementary""",11.46,4,75,40,389,0.0,56.150348,28.927687,21.269509
…,…,…,…,…,…,…,…,…,…
"""Milk (Harvey) Civil Rights Ele…",21.5,9,73,71,133,0.0,54.352953,52.617792,26.742686
"""Everett Middle""",15.2,79,1,4,404,61.228763,0.0,0.0,30.614381
"""Lilienthal (Claire) Elementary""",40.88,31,87,61,674,23.063878,69.158131,44.586996,39.968221
